In [2]:
# Data & analysis
import polars as pl
import numpy as np
from datetime import datetime, timedelta

# ML
import torch
import torch.nn as nn
import torch.optim as optim

# Visualization
import altair as alt

# Project modules (packaged)
from adausdt_qml import research, binance, models

In [3]:
# Research configuration
sym = "ADAUSDT"
time_interval = "16h"
max_lags = 4
forecast_horizon = 1

In [4]:
research.set_seed(99)

In [5]:
start_date = datetime(2023, 2, 8, 0, 0)
end_date   = datetime(2026, 2, 8, 0, 0)

In [6]:
ts = research.load_ohlc_timeseries_range(
    sym,
    time_interval,
    start_date,
    end_date
)

Loading ADAUSDT: 100%|██████████| 1097/1097 [00:20<00:00, 53.26day/s]


In [7]:
# target
ts = ts.with_columns(
    (pl.col("close") / pl.col("close").shift(forecast_horizon))
    .log()
    .alias("close_log_return")
)

target = "close_log_return"
lr = pl.col(target)

# Create direction column
ts = ts.with_columns(
    (pl.col("close_log_return") > 0).cast(pl.Int8).alias("return_dir")
)

# features
ts = ts.with_columns(
    lr.shift(forecast_horizon * 1).alias(f"{target}_lag_1"),
    lr.shift(forecast_horizon * 2).alias(f"{target}_lag_2"),
    lr.shift(forecast_horizon * 3).alias(f"{target}_lag_3"),
    lr.shift(forecast_horizon * 4).alias(f"{target}_lag_4"),
).drop_nulls()

## Train / test split (time-based)

Because this is a time series, we split **chronologically** (not randomly).  
The model is trained on the first part of the sample and evaluated on the most recent observations.

This avoids look-ahead bias and mimics a real trading workflow.

In [8]:
test_size = 0.25
int(len(ts) * test_size)
split_idx = int(len(ts) * (1-test_size))
ts_train, ts_test = ts[:split_idx], ts[split_idx:]

### Sanity check

We inspect the resulting subsets to confirm:
- The split is chronological (train dates come before test dates)
- There is no overlap

In [9]:
ts_train

datetime,open,high,low,close,close_log_return,return_dir,close_log_return_lag_1,close_log_return_lag_2,close_log_return_lag_3,close_log_return_lag_4
datetime[μs],f64,f64,f64,f64,f64,i8,f64,f64,f64,f64
2023-02-11 08:00:00,0.3593,0.3701,0.3586,0.3683,0.029203,1,-0.006965,-0.004985,-0.085721,0.004829
2023-02-12 00:00:00,0.3683,0.3708,0.3642,0.3697,0.003794,1,0.029203,-0.006965,-0.004985,-0.085721
2023-02-12 16:00:00,0.3697,0.3737,0.3591,0.3641,-0.015263,0,0.003794,0.029203,-0.006965,-0.004985
2023-02-13 08:00:00,0.3626,0.3627,0.3445,0.3589,-0.014385,0,-0.015263,0.003794,0.029203,-0.006965
2023-02-14 00:00:00,0.359,0.3819,0.3542,0.3789,0.054229,1,-0.014385,-0.015263,0.003794,0.029203
…,…,…,…,…,…,…,…,…,…,…
2025-05-08 00:00:00,0.6713,0.7435,0.6697,0.7313,0.085608,1,-0.010668,0.040454,-0.015381,-0.020789
2025-05-08 16:00:00,0.7313,0.7713,0.728,0.7693,0.050657,1,0.085608,-0.010668,0.040454,-0.015381
2025-05-09 08:00:00,0.7938,0.8181,0.7718,0.7771,0.010088,1,0.050657,0.085608,-0.010668,0.040454


In [10]:
ts_test

datetime,open,high,low,close,close_log_return,return_dir,close_log_return_lag_1,close_log_return_lag_2,close_log_return_lag_3,close_log_return_lag_4
datetime[μs],f64,f64,f64,f64,f64,i8,f64,f64,f64,f64
2025-05-11 08:00:00,0.7912,0.8161,0.781,0.8049,-0.043398,0,0.048005,0.030542,0.010088,0.050657
2025-05-12 00:00:00,0.8049,0.8641,0.7958,0.8216,0.020536,1,-0.043398,0.048005,0.030542,0.010088
2025-05-12 16:00:00,0.8215,0.8339,0.7796,0.8166,-0.006104,0,0.020536,-0.043398,0.048005,0.030542
2025-05-13 08:00:00,0.7912,0.8418,0.786,0.8297,0.015915,1,-0.006104,0.020536,-0.043398,0.048005
2025-05-14 00:00:00,0.8297,0.8336,0.7974,0.8069,-0.027864,0,0.015915,-0.006104,0.020536,-0.043398
…,…,…,…,…,…,…,…,…,…,…
2026-02-06 00:00:00,0.2451,0.2744,0.2204,0.2732,0.108538,1,-0.155025,-0.000699,-0.017651,-0.025069
2026-02-06 16:00:00,0.2732,0.2841,0.2722,0.2759,0.009834,1,0.108538,-0.155025,-0.000699,-0.017651
2026-02-07 08:00:00,0.2681,0.2796,0.2658,0.2719,-0.014604,0,0.009834,0.108538,-0.155025,-0.000699


We'll check the **Directional Balance** of the train and the test dataset

In [11]:
def directional_balance(df: pl.DataFrame, col: str) -> pl.DataFrame:
    counts = (
        df
        .select(pl.col(col).value_counts())
        .unnest(col)
    )

    return (
        counts
        .with_columns(
            (pl.col("count") / pl.col("count").sum()).alias("frequency")
        )
        .sort(col)
    )

In [12]:
train_balance = directional_balance(ts_train, "return_dir")
test_balance  = directional_balance(ts_test, "return_dir")

train_balance, test_balance

(shape: (2, 3)
 ┌────────────┬───────┬───────────┐
 │ return_dir ┆ count ┆ frequency │
 │ ---        ┆ ---   ┆ ---       │
 │ i8         ┆ u32   ┆ f64       │
 ╞════════════╪═══════╪═══════════╡
 │ 0          ┆ 629   ┆ 0.511382  │
 │ 1          ┆ 601   ┆ 0.488618  │
 └────────────┴───────┴───────────┘,
 shape: (2, 3)
 ┌────────────┬───────┬───────────┐
 │ return_dir ┆ count ┆ frequency │
 │ ---        ┆ ---   ┆ ---       │
 │ i8         ┆ u32   ┆ f64       │
 ╞════════════╪═══════╪═══════════╡
 │ 0          ┆ 217   ┆ 0.527981  │
 │ 1          ┆ 194   ┆ 0.472019  │
 └────────────┴───────┴───────────┘)

Data is **balanced** quite well.

## Feature Selection: Auto-Regressive Log-Returns

We construct features using a simple **auto-regressive (AR)** structure based on lagged log-returns of the closing price.

At this stage, four lagged returns are generated:

- `close_log_return_lag_1`
- `close_log_return_lag_2`
- `close_log_return_lag_3`
- `close_log_return_lag_4`

These features capture short-term price dynamics at the selected 16-hour frequency.

The choice of four lags defines a **feature pool**, not a final model specification.  
All combinations of these lagged features will be evaluated later using a systematic benchmarking procedure, and the final feature set will be selected **empirically based on out-of-sample performance**.

In [13]:
# Feature set used by the model
features = [
    f"{target}_lag_1",
    f"{target}_lag_2",
    f"{target}_lag_3",
    f"{target}_lag_4",
]

## Convert to PyTorch tensors

The model is implemented in PyTorch, so the data must be converted from
Polars DataFrames into PyTorch tensors.

- `X` represents the feature matrix with shape `(T, k)`
  - `T` = number of observations
  - `k` = number of lagged features
- `y` represents the target vector with shape `(T, 1)`

The target vector is explicitly reshaped to `(T, 1)` because PyTorch loss
functions expect a 2-dimensional target tensor.

In [14]:
X_train = torch.tensor(ts_train[features].to_numpy(), dtype=torch.float32)
X_test  = torch.tensor(ts_test[features].to_numpy(),  dtype=torch.float32)

y_train = torch.tensor(ts_train[target].to_numpy(), dtype=torch.float32).reshape(-1, 1)
y_test  = torch.tensor(ts_test[target].to_numpy(),  dtype=torch.float32).reshape(-1, 1)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([1230, 4]),
 torch.Size([1230, 1]),
 torch.Size([411, 4]),
 torch.Size([411, 1]))

## Model Specification (AR Model)

We start with a **linear regression model** implemented in PyTorch.

The model consists of a single linear layer that maps the selected lagged log-return features directly to the next-period log return. This setup corresponds to a standard **auto-regressive linear model**, estimated using gradient-based optimization rather than closed-form OLS.

This choice provides a transparent baseline that is easy to interpret and serves as a reference point for more complex models.

In [15]:
class LinearModel(nn.Module):
    def __init__(self, input_features):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_features, 1)

    def forward(self, x):
        return self.linear(x)

## Model Complexity 

The linear model is intentionally kept **minimal**.

It is an **AR(4) model**, where the next-period log return is modeled as a linear combination of the four most recent lagged log returns:

y(t+1) = w1 · x(t) + w2 · x(t−1) + w3 · x(t−2) + w4 · x(t−3) + b

This formulation makes the model fully interpretable: each weight measures the marginal contribution of a specific lag to the predicted return.

In [16]:
input_features = 4

linear_model = LinearModel(input_features)

research.print_model_info(linear_model, "Linear Model")
research.total_model_params(linear_model)


Linear Model

Architecture:
  LinearModel(
  (linear): Linear(in_features=4, out_features=1, bias=True)
)

Parameter Count:
  Total parameters:      5
  Trainable parameters:  5



5

## Training Setup

We train the linear AR model using **full-batch gradient descent** (i.e., we use the entire training set in each update).

**Loss:** Mean Absolute Error (L1)  
MAE = (1/n) * Σ|yᵢ - ȳ|

**Optimizer:** Adam  
**Goal:** minimize prediction error on the next-period log return.

In [17]:
# hyperparameters and learning rate
no_epochs = 1000 * 5
lr = 0.0005

# create model
model = LinearModel(len(features))

# loss function (could be also HuberLoss or MSE..)
criterion = nn.L1Loss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

## Training Loop

At each epoch we:
1. compute predictions on the training set  
2. compute the loss  
3. backpropagate gradients  
4. update parameters with Adam  

We print the training loss every fixed number of epochs to monitor convergence.

In [18]:
print("\nTraining model...")

for epoch in range(no_epochs):
    # forward pass
    y_hat = model(X_train)
    loss = criterion(y_hat, y_train)

    # backward pass
    optimizer.zero_grad()   # clear old gradients
    loss.backward()         # compute gradients
    optimizer.step()        # update weights

    train_loss = loss.item()

    if (epoch + 1) % 500 == 0:
        print(f"Epoch [{epoch+1}/{no_epochs}], Loss: {train_loss:.6f}")


Training model...
Epoch [500/5000], Loss: 0.025756
Epoch [1000/5000], Loss: 0.024210
Epoch [1500/5000], Loss: 0.023982
Epoch [2000/5000], Loss: 0.023969
Epoch [2500/5000], Loss: 0.023968
Epoch [3000/5000], Loss: 0.023968
Epoch [3500/5000], Loss: 0.023968
Epoch [4000/5000], Loss: 0.023968
Epoch [4500/5000], Loss: 0.023968
Epoch [5000/5000], Loss: 0.023968


## Learned Parameters and Out-of-Sample Evaluation

After training, we:
- print the learned weights and bias (interpretability)
- evaluate on the **held-out test set** (out-of-sample performance)

In [19]:
print("\nLearned parameters")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}:\n{param.data.numpy()}")

# Evaluation
model.eval()
with torch.no_grad():
    y_hat = model(X_test)
    test_loss = criterion(y_hat, y_test)
    print(f"\nTest Loss: {test_loss.item():.6f}, Train Loss: {train_loss:.6f}")


Learned parameters
linear.weight:
[[-0.07923951 -0.02594776 -0.01998974 -0.00465055]]
linear.bias:
[-0.00054805]

Test Loss: 0.024761, Train Loss: 0.023968


All lagged coefficients are **negative**, indicating a **mean-reversion structure** in log returns.

## Trading Performance Evaluation

After training the linear AR model, we evaluate its performance from a **trading perspective** rather than a pure prediction accuracy standpoint.

The model’s output is transformed into a **directional trading strategy**:
- Go **long** when the predicted return is positive  
- Go **short** when the predicted return is negative  

All calculations are performed using **log returns**, which are additive over time and naturally support compounding.

### Strategy Construction
- Trading signal: `sign(predicted return)`
- Trade log return: `signal × realized log return`
- Equity curve: cumulative sum of trade log returns

### Performance and Risk Metrics
We evaluate the strategy using the following indicators:
- Equity curve
- Maximum drawdown
- Win rate
- Expected value per trade
- Total and compounded return
- Volatility of trade returns
- Annualized Sharpe ratio

In [20]:
annualized_rate = research.sharpe_annualization_factor(time_interval, 365, 24)

In [21]:
# Trade-level results
trade_results = pl.DataFrame({
    'y_hat': y_hat.squeeze(),
    'y': y_test.squeeze()
}).with_columns(
    (pl.col('y_hat').sign() == pl.col('y').sign()).alias('is_won'),
    pl.col('y_hat').sign().alias('signal'),
).with_columns(
    (pl.col('signal') * pl.col('y')).alias('trade_log_return')
).with_columns(
    pl.col('trade_log_return').cum_sum().alias('equity_curve')
)

# Drawdown (log space)
trade_results = trade_results.with_columns(
    (pl.col('equity_curve') - pl.col('equity_curve').cum_max()).alias('drawdown_log')
)

max_drawdown_log = trade_results['drawdown_log'].min()

# Win rate
win_rate = trade_results['is_won'].mean()

# Expected value
avg_win = trade_results.filter(pl.col('is_won'))['trade_log_return'].mean()
avg_loss = trade_results.filter(~pl.col('is_won'))['trade_log_return'].mean()
ev = win_rate * avg_win + (1 - win_rate) * avg_loss

# Total and compounded return
total_log_return = trade_results['trade_log_return'].sum()
compound_return = np.exp(total_log_return)

# Risk metrics
equity_trough = trade_results['equity_curve'].min()
equity_peak = trade_results['equity_curve'].max()
std = trade_results['trade_log_return'].std()

# Annualized Sharpe ratio
sharpe = ev / std * annualized_rate

In [22]:
metrics = pl.DataFrame({
    "Metric": [
        "Win Rate",
        "Expected Value (EV)",
        "Total Log Return",
        "Compound Return",
        "Max Drawdown (log)",
        "Equity Peak",
        "Equity Trough",
        "Return Std Dev",
        "Annualized Sharpe"
    ],
    "Value": [
        win_rate,
        ev,
        total_log_return,
        compound_return,
        max_drawdown_log,
        equity_peak,
        equity_trough,
        std,
        sharpe
    ]
})

metrics

Metric,Value
str,f64
"""Win Rate""",0.523114
"""Expected Value (EV)""",0.001922
"""Total Log Return""",0.790041
"""Compound Return""",2.203487
"""Max Drawdown (log)""",-0.495437
"""Equity Peak""",1.112434
"""Equity Trough""",-0.057324
"""Return Std Dev""",0.03487
"""Annualized Sharpe""",1.289875


In [23]:
# Plot equity curve
research.plot_column(trade_results, 'equity_curve')

alt.Chart(...)

## Feature Selection via Built-in Benchmarking

To quickly identify the best-performing linear specification, we use the **built-in benchmarking utilities** from the local `research` library.

The function evaluates **all combinations** of lagged features (up to a chosen maximum) and ranks models by **out-of-sample Sharpe ratio** computed on the **test split**.

In [24]:
no_lags = 4
feature_pool = [f"{target}_lag_{i}" for i in range(1, no_lags + 1)]

pl.Config.set_fmt_str_lengths(200)

benchmarks = research.benchmark_linear_models(
    ts.drop_nulls(),
    target,
    feature_pool,
    annualized_rate,
    max_no_features=3, 
    loss=nn.L1Loss(), # you can change the loss function here 
    test_size=0.25
)

benchmarks

features,target,no_trades,win_rate,avg_win,avg_loss,best_trade,worst_trade,ev,std,total_log_return,compound_return,max_drawdown,equity_trough,equity_peak,sharpe,weights,biases
str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
"""close_log_return_lag_1""","""close_log_return""",411,0.545012,0.026433,-0.022882,0.250036,-0.102326,0.003995,0.034693,1.642048,5.16574,-0.50213,-0.027851,1.642048,2.69458,"""[-0.08151469]""","""-0.0009305799030698836"""
"""close_log_return_lag_3""","""close_log_return""",411,0.542579,0.026383,-0.02296,0.250036,-0.108538,0.003812,0.034714,1.566834,4.791454,-0.506719,-0.033689,1.692057,2.569626,"""[-0.02666365]""","""-0.0007373386761173606"""
"""close_log_return_lag_1,close_log_return_lag_4""","""close_log_return""",411,0.542579,0.026266,-0.023099,0.250036,-0.102326,0.003686,0.034728,1.514888,4.548911,-0.50213,-0.125169,1.514887,2.483455,"""[-0.08500567 0.00123597]""","""-0.0008204468176700175"""
"""close_log_return_lag_3,close_log_return_lag_4""","""close_log_return""",411,0.537713,0.026093,-0.023333,0.250036,-0.155025,0.003244,0.034772,1.333239,3.793309,-0.474445,0.022862,1.672646,2.182888,"""[-0.02874127 -0.00766001]""","""-0.0006995085859671235"""
"""close_log_return_lag_2,close_log_return_lag_3,close_log_return_lag_4""","""close_log_return""",411,0.542579,0.025673,-0.023802,0.250036,-0.155025,0.003042,0.03479,1.250262,3.491256,-0.474445,0.013052,1.519862,2.045956,"""[-0.00480906 -0.02746062 -0.00801369]""","""-0.0008177283452823758"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""close_log_return_lag_1,close_log_return_lag_3,close_log_return_lag_4""","""close_log_return""",411,0.518248,0.026232,-0.023295,0.250036,-0.155025,0.002372,0.034842,0.97503,2.651247,-0.417762,0.027312,1.300939,1.593171,"""[-0.08057362 -0.02588925 -0.0018502 ]""","""-0.0004342469619587064"""
"""close_log_return_lag_2""","""close_log_return""",411,0.532847,0.025251,-0.024323,0.250036,-0.108538,0.002092,0.03486,0.859897,2.362917,-0.593639,-0.14011,0.936243,1.404321,"""[-0.01136332]""","""-0.0005409722216427326"""
"""close_log_return_lag_1,close_log_return_lag_2,close_log_return_lag_4""","""close_log_return""",411,0.508516,0.02599,-0.023604,0.250036,-0.155025,0.001615,0.034886,0.663731,1.942024,-0.541537,0.0094,1.079797,1.083167,"""[-0.08177126 -0.02623206 -0.00688894]""","""-0.0005106196040287614"""


In [25]:
features = ['close_log_return_lag_1']
model = LinearModel(len(features))
model_trades = research.learn_model_trades(ts.drop_nulls(), features, target, model, loss=nn.L1Loss())
research.plot_column(model_trades, 'equity_curve')

alt.Chart(...)

## Final Model Selection and Saving Weights

Based on the benchmarking results, we select the model with the **highest out-of-sample Sharpe ratio (≈ 2.94)**.

The optimal specification uses a **sparse AR structure**, retaining only the most informative lags:
- `close_log_return_lag_1`
- `close_log_return_lag_3`


In [26]:
torch.save(model.state_dict(), 'model_weights.pth')

## Adding Transaction Fees

To make results more realistic, we include **transaction costs**.  
We assume we pay the **taker fee** on entry and again on exit (a *round-trip*), so the fee is applied twice.

Then we compute the **net equity curve** by cumulatively summing the net log returns.

In [27]:
# Transaction fees (example, I took these values from Hyperliquid)
maker_fee = 0.00015
taker_fee = 0.00035

# Round-trip taker fee in log space (enter + exit)
roundtrip_fee_log = float(np.log(1 - 2 * taker_fee))

model_trades = model_trades.with_columns(
    pl.lit(roundtrip_fee_log).alias("tx_fee_log"),
    (pl.col("trade_log_return") + pl.lit(roundtrip_fee_log)).alias("trade_log_return_net"),
    (pl.col("trade_log_return") + pl.lit(roundtrip_fee_log)).cum_sum().alias("equity_curve_net"),
)

model_trades

y_pred,y_true,is_won,position,trade_log_return,equity_curve,drawdown_log_return,tx_fee_log,trade_log_return_net,equity_curve_net
f32,f32,bool,f32,f32,f32,f32,f64,f32,f32
-0.004849,-0.043398,true,-1.0,0.043398,0.043398,0.0,-0.0007,0.042698,0.042698
0.002623,0.020536,true,1.0,0.020536,0.063933,0.0,-0.0007,0.019835,0.062533
-0.002603,-0.006104,true,-1.0,0.006104,0.070038,0.0,-0.0007,0.005404,0.067937
-0.000426,0.015915,false,-1.0,-0.015915,0.054123,-0.015915,-0.0007,-0.016615,0.051322
-0.002226,-0.027864,true,-1.0,0.027864,0.081987,0.0,-0.0007,0.027164,0.078486
…,…,…,…,…,…,…,…,…,…
0.011748,0.108538,true,1.0,0.108538,1.625116,0.0,-0.0007,0.107838,1.340117
-0.009797,0.009834,false,-1.0,-0.009834,1.615282,-0.009834,-0.0007,-0.010535,1.329582
-0.001729,-0.014604,true,-1.0,0.014604,1.629886,0.0,-0.0007,0.013904,1.343486


In [28]:
research.plot_column(model_trades, 'equity_curve_net')

alt.Chart(...)